In [1]:
from gensim.models import AuthorTopicModel

model = AuthorTopicModel.load('model.atmodel')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
for topic in model.show_topics(num_topics=100):
    words = ''
    for word, prob in model.show_topic(topic[0]):
        words += word + ' '
    print('Words: ' + words)
    print()

Words: encarcela cuñada simulación populismo lesbiana voltaire borja alison falsamente esclavo 

Words: Quadratín denuncia quadratín dispareja links saliendo topó dirigió celebren causaron sports 

Words: mesaparatodos marcaclaromvs entrevistamvs permanecerá opiniónmvs pupilentes pregón despertaremos convenga suplemento 

Words: magazinesd afianzar etanol cuchillo quirino enterrado ordaz chamaqueó incumplidas tortuga 

Words: summer primerasplanas hoyenlamúsica primerasplanasciudad primerasplanascultura MILENIO DIARIO primerasplanasdeportes oncenoticias primerasplanasespectaculos primerasplanaseconomia 

Words: compositor demostraron #EnPortada malversación reflectores cerrarle escénico Corazón de León comparación necios 

Words: poderes parker warholmuseojumex Peter Parker femenino entres escandón comprarlo queretaro Gobierno de Corea del Sur 

Words: jajaja best ever movie alone let homosexualidad fact dif summit 

Words: health watergate league which fallen garcés Santiago Garcés ev

Words: leaks @Javier_Duarte… receso Sistema Nacional Anticorrupción... ¿Generación Sayuri Herrera Román merezco … Ahorita “Paciencia ale 

Words: zamorano José López Zamorano Arnoldo Cuellar Zócalo” de la matanzas cuellar pensamos religión delictivas Pablo Palomo Reyna 

Words: denunciaenuno laferia mándanos @DavidRubi_ ► Onda Tropical Número Tiempo Real appsdesalud AlertaVial Se chief LaFeria en @ElFinanciero_Mx Buen 

Words: a y méxico el cdmx duarte gobierno opinión eu javier 

Words: lsrveracruz LSRVeracruz Prisión preventiva FOTOS Mia VIDEO Mujeres lampard YoSoiTu Casa de Lampard FOTOS Maribel VIDEO Presentadora de TV contenía pastilla 

Words: pegasu desacuerdos abandonado gabinetes lomásescuchado Ninel Conde cojea grasas rector benevolente 

Words: EN DIRECTO pm blog big escrito back guíe tacos regala war 

Words: deportesw marthadebayleenw asísopitas elfutbolentusoídos #Nacional opiniónlsr #AsíSopitas sustituido Francia ! AsíSopitas Así 

Words: campusnewsmilenio tontos drake i

In [3]:
model.show_topics(num_topics=100)

[(0,
  '0.010*"encarcela" + 0.010*"cuñada" + 0.007*"simulación" + 0.005*"populismo" + 0.005*"lesbiana" + 0.005*"voltaire" + 0.005*"borja" + 0.005*"alison" + 0.005*"falsamente" + 0.005*"esclavo"'),
 (1,
  '0.013*"Quadratín denuncia" + 0.013*"quadratín" + 0.012*"dispareja" + 0.011*"links" + 0.011*"saliendo" + 0.011*"topó" + 0.011*"dirigió" + 0.009*"celebren" + 0.009*"causaron" + 0.008*"sports"'),
 (2,
  '0.101*"mesaparatodos" + 0.027*"marcaclaromvs" + 0.018*"entrevistamvs" + 0.012*"permanecerá" + 0.011*"opiniónmvs" + 0.008*"pupilentes" + 0.008*"pregón" + 0.008*"despertaremos" + 0.007*"convenga" + 0.007*"suplemento"'),
 (3,
  '0.027*"magazinesd" + 0.014*"afianzar" + 0.014*"etanol" + 0.011*"cuchillo" + 0.010*"quirino" + 0.007*"enterrado" + 0.007*"ordaz" + 0.007*"chamaqueó" + 0.007*"incumplidas" + 0.007*"tortuga"'),
 (4,
  '0.015*"summer" + 0.015*"primerasplanas" + 0.014*"hoyenlamúsica" + 0.014*"primerasplanasciudad" + 0.014*"primerasplanascultura" + 0.011*"MILENIO DIARIO" + 0.011*"primeras

In [4]:
%%time
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
smallest_author = 0  # Ignore authors with documents less than this.
authors = [model.author2id[a] for a in model.author2id.keys() if len(model.author2doc[a]) >= smallest_author]
_ = tsne.fit_transform(model.state.gamma[authors, :])  # Result stored in tsne.embedding_

Wall time: 1.04 s


In [5]:

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [6]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, ColumnDataSource

x = tsne.embedding_[:, 0]
y = tsne.embedding_[:, 1]
author_names = [model.id2author[a] for a in authors]

scale = 0.01
author_sizes = [len(model.author2doc[a]) for a in author_names]
radii = [size * scale for size in author_sizes]

source = ColumnDataSource(
        data=dict(
            x=x,
            y=y,
            author_names=author_names,
            author_sizes=author_sizes,
            radii=radii,
        )
    )

hover = HoverTool(
        tooltips=[
        ("author", "@author_names"),
        ("size", "@author_sizes"),
        ]
    )

p = figure(tools=[hover, 'crosshair,pan,wheel_zoom,box_zoom,reset,save,lasso_select'])
p.scatter('x', 'y', radius='radii', source=source, fill_alpha=0.6, line_color=None)
show(p)

In [7]:
from gensim.similarities import MatrixSimilarity


index = MatrixSimilarity(model[list(model.id2author.values())])



In [8]:

from gensim import matutils
import pandas as pd


author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

def similarity(vec1, vec2):
    dist = matutils.hellinger(matutils.sparse2full(vec1, model.num_topics), \
                              matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    return sim

def get_sims(vec):
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims

def get_table(name, top_n=10, smallest_author=1):
    sims = get_sims(model.get_author_topics(name))


    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            table.append((author_name, sim, author_size))
            

    df = pd.DataFrame(table, columns=['Author', 'Score', 'Size'])
    df = df.sort_values('Score', ascending=False)[:top_n]
    
    return df

In [9]:
get_table('Pajaropolitico',top_n=136)

,Author,Score,Size
49,Pajaropolitico,1.000000,820
67,UnoNoticias,0.760088,516
33,JoseCardenas1,0.754799,339
34,LaRazon_mx,0.753353,793
24,ExpansionMx,0.752719,1371
129,sdpnoticias,0.751548,55
107,lacronicadehoy,0.750305,832
111,lopezdoriga,0.748428,836
61,SanjuanaMtz,0.745505,54
131,sopitas,0.733811,286


In [11]:
import pickle
dictionary = pickle.load(open("dictionary.p", "rb"))

In [11]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(model, model.corpus,dictionary)